In [1]:
%%javascript

window.load_remote_theme = true
var theme_js = "https://odhk.github.io/hyrule_theme/custom.js";

window.load_local_theme = function(){
    var hostname = document.location.hostname
    return ((hostname == "localhost" || hostname == '127.0.0.1') && !load_remote_theme)
}

var url = load_local_theme() ? document.location.origin + "/files/theme/custom.js" : theme_js

$.getScript(url)

<IPython.core.display.Javascript object>

In [4]:
import numpy as np
import pandas as pd
import gc
import lightgbm as lgb
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import random
import os

In [47]:
X = pd.read_pickle('top_feature1000.pkl')

sub_sample = pd.read_csv('3rd_security_submit_sample.csv')

label = pd.read_pickle('label.pkl')
label = label[0].values

X['id'] = X.iloc[:, 0].values

X.drop('file_id', axis = 1, inplace = True)

train_FE = X.iloc[:len(label),:]
test_FE = X.iloc[len(label):,:]

train_FE['label'] = label

test_FE.drop('id', axis = 1, inplace = True)

x0 = train_FE[train_FE['label'] == 0]

x1 = train_FE[train_FE['label'] != 0]

x0.shape, x1.shape

In [5]:
index = pd.read_csv('index.csv')

In [7]:
params = {"objective": "multiclass",
          "boosting_type": "gbdt",
          "learning_rate": 0.05,
          "num_leaves": 15,
          # "max_bin": 128,
          "feature_fraction": 0.85,
#           "min_child_samples": 10,
#           "min_child_weight": 150,
#           "min_split_gain": 0,
          "subsample": 0.85,
          #'metric':'logloss',
           'lambda_l1':0.02,
           'lambda_l2':0.02,
          'seed':666,
          'num_class':6
          }

In [ ]:
loss_list = []
pred = pd.DataFrame()
for i in range(10):
    print('***********第%s次抽样***********'%i)
    x0_part = x0.copy()
    x0_part.index = range(x0_part.shape[0])
    x1.index = range(x1.shape[0])
    permutation = np.random.permutation(x0_part.shape[0])
    
    ratio = int(x0.shape[0]*random.uniform(1.5,5)/10)

    select = x0_part.iloc[permutation[:ratio], :]
    
    print('负样本抽样数量：')
    print(select.shape)
    
    data_new = pd.concat([x1, select], axis = 0)
    
    permutation = np.random.permutation(data_new.shape[0])

    data_new = data_new.iloc[permutation, :]
    
    label_new = data_new['label'].values
    
    data_new = data_new.drop(['label', 'id'], axis = 1)
    
    dtrain = lgb.Dataset(data_new, label=label_new)

    hist = lgb.cv(params, dtrain, num_boost_round = 1000, verbose_eval=20, early_stopping_rounds=30, stratified=False)
    
    print('最优结果%s'%hist['multi_logloss-mean'][-1])
    loss_list.append(hist['multi_logloss-mean'][-1])
    print('总loss%s'%loss_list)
    model = lgb.train(params=params, train_set=dtrain, num_boost_round=int(1.1*len(hist['multi_logloss-mean'])), verbose_eval=10,\
                      valid_sets=dtrain)
    
    pred_temp = pd.DataFrame(model.predict(test_FE))
    
    pred = pd.concat([pred, pred_temp], axis = 1)

***********第0次抽样***********
负样本抽样数量：
(53670, 1001)
[20]	cv_agg's multi_logloss: 0.537977 + 0.000852458
[220]	cv_agg's multi_logloss: 0.0582876 + 0.00358294
最优结果0.05803473436927152
总loss[0.02618883638668918, 0.03287141226624521, 0.051883784445321234, 0.02758720799855078, 0.05803473436927152]
[10]	training's multi_logloss: 0.941344
[20]	training's multi_logloss: 0.569703
[30]	training's multi_logloss: 0.364831
[40]	training's multi_logloss: 0.243211
[50]	training's multi_logloss: 0.168723
[60]	training's multi_logloss: 0.122148
[70]	training's multi_logloss: 0.0915753
[80]	training's multi_logloss: 0.0708085
[90]	training's multi_logloss: 0.0563478
[100]	training's multi_logloss: 0.0458895
[110]	training's multi_logloss: 0.0379351
[120]	training's multi_logloss: 0.0316158
[130]	training's multi_logloss: 0.0267317
[140]	training's multi_logloss: 0.022903
[150]	training's multi_logloss: 0.0198327
[160]	training's multi_logloss: 0.0173106
[170]	training's multi_logloss: 0.0152548
[180]	trai

In [22]:
pred

,0,1,2,3,4,5,0,1,2,3,...,2,3,4,5,0,1,2,3,4,5
0,0.999581,0.000019,0.000090,0.000037,0.000014,0.000258,0.999388,0.000020,0.000065,0.000051,...,0.000220,0.000095,0.000037,0.000832,0.998179,0.000060,0.000258,0.000137,0.000042,0.001325
1,0.998742,0.000117,0.000580,0.000050,0.000021,0.000489,0.998390,0.000081,0.000728,0.000091,...,0.002001,0.000223,0.000068,0.001601,0.997043,0.000205,0.001350,0.000180,0.000048,0.001174
2,0.999794,0.000018,0.000042,0.000056,0.000010,0.000079,0.999755,0.000019,0.000043,0.000082,...,0.000076,0.000181,0.000024,0.000121,0.999548,0.000028,0.000084,0.000179,0.000023,0.000138
3,0.999773,0.000011,0.000045,0.000022,0.000010,0.000139,0.999636,0.000013,0.000048,0.000029,...,0.000085,0.000067,0.000022,0.000489,0.999588,0.000024,0.000078,0.000047,0.000020,0.000242
4,0.999911,0.000005,0.000028,0.000008,0.000005,0.000042,0.999915,0.000006,0.000029,0.000008,...,0.000055,0.000020,0.000011,0.000087,0.999832,0.000011,0.000061,0.000017,0.000010,0.000069
5,0.999383,0.000015,0.000509,0.000021,0.000014,0.000059,0.999491,0.000015,0.000405,0.000021,...,0.000721,0.000039,0.000027,0.000073,0.999197,0.000031,0.000616,0.000039,0.000029,0.000087
6,0.999578,0.000017,0.000219,0.000027,0.000014,0.000144,0.999400,0.000021,0.000351,0.000035,...,0.000292,0.000054,0.000031,0.000218,0.999354,0.000052,0.000271,0.000050,0.000027,0.000246
7,0.999919,0.000008,0.000017,0.000016,0.000006,0.000034,0.999906,0.000007,0.000027,0.000017,...,0.000041,0.000032,0.000013,0.000049,0.999889,0.000012,0.000026,0.000023,0.000009,0.000040
8,0.998270,0.000065,0.000255,0.000115,0.000037,0.001257,0.997231,0.000093,0.000299,0.000169,...,0.000671,0.000280,0.000125,0.004492,0.994173,0.000116,0.000500,0.000261,0.000091,0.004860
9,0.996067,0.000374,0.001832,0.000512,0.000148,0.001067,0.995041,0.000401,0.002290,0.000700,...,0.007230,0.001693,0.000569,0.002651,0.992204,0.000334,0.004588,0.000983,0.000333,0.001558


In [25]:
p = pd.DataFrame(sub_sample.file_id)

p['prob0'] = pred[0].mean(axis = 1)
p['prob1'] = pred[1].mean(axis = 1)
p['prob2'] = pred[2].mean(axis = 1)
p['prob3'] = pred[3].mean(axis = 1)
p['prob4'] = pred[4].mean(axis = 1)
p['prob5'] = pred[5].mean(axis = 1)

In [29]:
p.to_csv('/home/libo/Security/sub/8.22_10_sampling.csv', index = None ,encoding = 'utf-8')

In [44]:
p = pd.DataFrame(sub_sample.file_id)

p['prob0'] = pred.iloc[:,-6:][0]
p['prob1'] = pred.iloc[:,-6:][1]
p['prob2'] = pred.iloc[:,-6:][2]
p['prob3'] = pred.iloc[:,-6:][3]
p['prob4'] = pred.iloc[:,-6:][4]
p['prob5'] = pred.iloc[:,-6:][5]

In [46]:
p.to_csv('/home/libo/Security/sub/采样8.22.csv', index = None ,encoding = 'utf-8')

In [43]:
pred.iloc[:,-6:]

,0,1,2,3,4,5
0,0.998179,0.000060,0.000258,0.000137,0.000042,0.001325
1,0.997043,0.000205,0.001350,0.000180,0.000048,0.001174
2,0.999548,0.000028,0.000084,0.000179,0.000023,0.000138
3,0.999588,0.000024,0.000078,0.000047,0.000020,0.000242
4,0.999832,0.000011,0.000061,0.000017,0.000010,0.000069
5,0.999197,0.000031,0.000616,0.000039,0.000029,0.000087
6,0.999354,0.000052,0.000271,0.000050,0.000027,0.000246
7,0.999889,0.000012,0.000026,0.000023,0.000009,0.000040
8,0.994173,0.000116,0.000500,0.000261,0.000091,0.004860
9,0.992204,0.000334,0.004588,0.000983,0.000333,0.001558


In [35]:
pd.DataFrame(np.corrcoef(pred[0], rowvar = 0))

,0,1,2,3,4,5,6,7,8,9
0,1.000000,0.998995,0.997458,0.999229,0.995560,0.995657,0.998620,0.998792,0.997231,0.997433
1,0.998995,1.000000,0.997954,0.998910,0.996312,0.996543,0.998944,0.998966,0.998017,0.997774
2,0.997458,0.997954,1.000000,0.997542,0.998286,0.998402,0.998537,0.998050,0.998647,0.998878
3,0.999229,0.998910,0.997542,1.000000,0.995949,0.995738,0.998690,0.998673,0.997415,0.997602
4,0.995560,0.996312,0.998286,0.995949,1.000000,0.998316,0.997135,0.996792,0.998112,0.997894
5,0.995657,0.996543,0.998402,0.995738,0.998316,1.000000,0.997371,0.996697,0.998051,0.998293
6,0.998620,0.998944,0.998537,0.998690,0.997135,0.997371,1.000000,0.999025,0.998329,0.998540
7,0.998792,0.998966,0.998050,0.998673,0.996792,0.996697,0.999025,1.000000,0.997853,0.998068
8,0.997231,0.998017,0.998647,0.997415,0.998112,0.998051,0.998329,0.997853,1.000000,0.998517
9,0.997433,0.997774,0.998878,0.997602,0.997894,0.998293,0.998540,0.998068,0.998517,1.000000


In [66]:
X = pd.read_pickle('top_feature1000.pkl')
X = X.iloc[:,:300]
sub_sample = pd.read_csv('3rd_security_submit_sample.csv')

label = pd.read_pickle('label.pkl')
label = label[0].values

X['id'] = X.iloc[:, 0].values

X.drop('file_id', axis = 1, inplace = True)

In [67]:
train_FE = X.iloc[:len(label),:]
test_FE = X.iloc[len(label):,:]

train_FE['label'] = label

test_FE.drop('id', axis = 1, inplace = True)

/home/libo/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [68]:
train_FE.shape, test_FE.shape

((116624, 300), (53093, 299))

In [69]:
x0 = train_FE[train_FE['label'] == 0]

x1 = train_FE[train_FE['label'] != 0]

x0.shape, x1.shape

((111545, 300), (5079, 300))

In [70]:
index = pd.read_csv('index.csv')

In [71]:
params = {"objective": "multiclass",
          "boosting_type": "gbdt",
          "learning_rate": 0.05,
          "num_leaves": 5,
          # "max_bin": 128,
          "feature_fraction": 0.85,
#           "min_child_samples": 10,
#           "min_child_weight": 150,
#           "min_split_gain": 0,
          "subsample": 0.85,
          #'metric':'logloss',
#            'lambda_l1':0.02,
#            'lambda_l2':0.02,
          'seed':666,
          'num_class':6
          }

In [106]:
random.seed(2)
int(x0.shape[0]*random.uniform(1.5,5)/10)

54056

In [ ]:
loss_list = []
pred = pd.DataFrame()
for i in range(15):
    print('***********第%s次抽样***********'%i)
    x0_part = x0.copy()
    x0_part.index = range(x0_part.shape[0])
    x1.index = range(x1.shape[0])
    permutation = np.random.permutation(x0_part.shape[0])
    
    ratio = int(x0.shape[0]*random.uniform(1.5,5)/10)

    select = x0_part.iloc[permutation[:ratio], :]
    
    print('负样本抽样数量：')
    print(select.shape)
    
    data_new = pd.concat([x1, select], axis = 0)
    
    permutation = np.random.permutation(data_new.shape[0])

    data_new = data_new.iloc[permutation, :]
    
    label_new = data_new['label'].values
    
    data_new = data_new.drop(['label', 'id'], axis = 1)
    
    dtrain = lgb.Dataset(data_new, label=label_new)

    hist = lgb.cv(params, dtrain, num_boost_round = 1000, verbose_eval=20, early_stopping_rounds=30, stratified=False)
    
    print('最优结果%s'%hist['multi_logloss-mean'][-1])
    loss_list.append(hist['multi_logloss-mean'][-1])
    print('总loss%s'%loss_list)
    model = lgb.train(params=params, train_set=dtrain, num_boost_round=int(1.1*len(hist['multi_logloss-mean'])), verbose_eval=10,\
                      valid_sets=dtrain)
    
    pred_temp = pd.DataFrame(model.predict(test_FE))
    
    pred = pd.concat([pred, pred_temp], axis = 1)

***********第0次抽样***********
负样本抽样数量：
(24715, 300)
[20]	cv_agg's multi_logloss: 0.615233 + 0.00358708
[40]	cv_agg's multi_logloss: 0.292297 + 0.00414274
[60]	cv_agg's multi_logloss: 0.170287 + 0.00412122
[80]	cv_agg's multi_logloss: 0.118341 + 0.00405836
[100]	cv_agg's multi_logloss: 0.0933328 + 0.00398645
[120]	cv_agg's multi_logloss: 0.0793913 + 0.00393392
[140]	cv_agg's multi_logloss: 0.0707632 + 0.00410048
[160]	cv_agg's multi_logloss: 0.0650659 + 0.00422598
[180]	cv_agg's multi_logloss: 0.0610856 + 0.0042795
[200]	cv_agg's multi_logloss: 0.0581965 + 0.00441624
[220]	cv_agg's multi_logloss: 0.0560443 + 0.00442188
[240]	cv_agg's multi_logloss: 0.0543384 + 0.00452162
[260]	cv_agg's multi_logloss: 0.0529656 + 0.00463049
[280]	cv_agg's multi_logloss: 0.0519329 + 0.00470479
[300]	cv_agg's multi_logloss: 0.051075 + 0.00480555
[320]	cv_agg's multi_logloss: 0.0503055 + 0.00489633
[340]	cv_agg's multi_logloss: 0.0496798 + 0.00497883
[360]	cv_agg's multi_logloss: 0.0491524 + 0.00508604
[380]	

In [76]:
p = pd.DataFrame(sub_sample.file_id)

p['prob0'] = pred[0].mean(axis = 1)
p['prob1'] = pred[1].mean(axis = 1)
p['prob2'] = pred[2].mean(axis = 1)
p['prob3'] = pred[3].mean(axis = 1)
p['prob4'] = pred[4].mean(axis = 1)
p['prob5'] = pred[5].mean(axis = 1)

In [78]:
p.mean()

file_id    26546.000000
prob0          0.900665
prob1          0.007622
prob2          0.014091
prob3          0.012161
prob4          0.001237
prob5          0.064224
dtype: float64

In [79]:
p.to_csv('/home/libo/Security/sub/采样8.23.csv', index = None ,encoding = 'utf-8')

In [80]:
x1 = pd.read_csv('/home/libo/Security/sub/采样8.22.csv')
x2 = pd.read_csv('/home/libo/Security/sub/采样8.23.csv')

In [86]:
np.corrcoef([x1['prob0'], x2['prob0'], pred['prob0']], rowvar=1)

array([[1.        , 0.99771934, 0.99943215],
       [0.99771934, 1.        , 0.99942719],
       [0.99943215, 0.99942719, 1.        ]])

In [83]:
pred = x1.copy()

In [84]:
pred['prob0'] = 0.5*x1['prob0'] + 0.5*x2['prob0']
pred['prob1'] = 0.5*x1['prob1'] + 0.5*x2['prob1']
pred['prob2'] = 0.5*x1['prob2'] + 0.5*x2['prob2']
pred['prob3'] = 0.5*x1['prob3'] + 0.5*x2['prob3']
pred['prob4'] = 0.5*x1['prob4'] + 0.5*x2['prob4']
pred['prob5'] = 0.5*x1['prob5'] + 0.5*x2['prob5']

In [87]:
pred.to_csv('/home/libo/Security/sub/采样融合.csv', index = None ,encoding = 'utf-8')

#### stacking bagging

In [5]:
sub_sample = pd.read_csv('/home/libo/Security/3rd_security_submit_sample.csv')

label = pd.read_pickle('/home/libo/Security/label.pkl')
label = label[0].values

feature = pd.read_pickle('/home/libo/Security/top_feature1000.pkl')

feature['id'] = feature.iloc[:, 0].values

feature.drop('file_id', axis = 1, inplace = True)

train_FE = feature.iloc[:len(label),:]
test_FE = feature.iloc[len(label):,:]

train_FE.shape, test_FE.shape

path = '/home/libo/Security/stack'
os.chdir(path)

list_ = os.listdir()
train_list = [x for x in list_ if 'train' in x]
test_list = [x for x in list_ if 'test' in x]

len(train_list) == len(test_list)

len(train_list)

In [12]:
train_meta = pd.DataFrame()
for i in train_list:
    train_meta = pd.concat([train_meta, pd.read_csv(i)], axis = 1)
    print(train_meta.shape)

(116624, 6)
(116624, 12)
(116624, 18)
(116624, 24)
(116624, 30)
(116624, 36)
(116624, 42)
(116624, 48)
(116624, 54)
(116624, 60)
(116624, 66)
(116624, 72)
(116624, 78)
(116624, 84)
(116624, 90)
(116624, 96)
(116624, 102)
(116624, 192)
(116624, 198)
(116624, 204)
(116624, 210)
(116624, 216)
(116624, 222)
(116624, 228)
(116624, 234)
(116624, 240)
(116624, 246)
(116624, 252)
(116624, 258)


In [13]:
test_meta = pd.DataFrame()
for i in test_list:
    try:
        x = pd.read_csv(i).drop('file_id', axis = 1)
    except:
        x = pd.read_csv(i)
    test_meta = pd.concat([test_meta, x], axis = 1)
    print(test_meta.shape)

(53093, 6)
(53093, 12)
(53093, 18)
(53093, 24)
(53093, 30)
(53093, 36)
(53093, 42)
(53093, 48)
(53093, 54)
(53093, 60)
(53093, 66)
(53093, 72)
(53093, 78)
(53093, 84)
(53093, 90)
(53093, 96)
(53093, 102)
(53093, 108)
(53093, 114)
(53093, 120)
(53093, 126)
(53093, 132)
(53093, 138)
(53093, 144)
(53093, 150)
(53093, 156)
(53093, 162)
(53093, 168)
(53093, 258)


In [14]:
test_meta = test_meta[train_meta.columns]
test_meta.columns == train_meta.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [15]:
train_meta.shape, test_meta.shape

((116624, 258), (53093, 258))

In [16]:
test_FE.index = range(test_FE.shape[0])
test_meta.index = range(test_meta.shape[0])

In [17]:
params = {"objective": "multiclass",
          "boosting_type": "gbdt",
          "learning_rate": 0.05,
          "num_leaves": 3,
          # "max_bin": 128,
          "feature_fraction": 0.85,
#           "min_child_samples": 10,
#           "min_child_weight": 150,
#           "min_split_gain": 0,
          "subsample": 0.85,
          #'metric':'logloss',
#            'lambda_l1':0.1,
#            'lambda_l2':0.1,
          'seed':666,
          'num_class':6
          }

In [18]:
train_FE['label'] = label

test_FE.drop('id', axis = 1, inplace = True)

train_FE.shape, test_FE.shape

/home/libo/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/libo/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


((116624, 1001), (53093, 999))

In [19]:
train_FE = pd.concat([train_FE, train_meta], axis = 1)
test_FE = pd.concat([test_FE, test_meta], axis = 1)

train_FE.shape, test_FE.shape

In [21]:
np.sum(train_FE.drop(['id', 'label'], axis = 1).columns == test_FE.columns)

1257

In [22]:
x0 = train_FE[train_FE['label'] == 0]

x1 = train_FE[train_FE['label'] != 0]

x0.shape, x1.shape

((111545, 1259), (5079, 1259))

In [23]:
index = pd.read_csv('/home/libo/Security/index.csv')

In [24]:
params = {"objective": "multiclass",
          "boosting_type": "gbdt",
          "learning_rate": 0.05,
          "num_leaves": 3,
          # "max_bin": 128,
          "feature_fraction": 0.85,
#           "min_child_samples": 10,
#           "min_child_weight": 150,
#           "min_split_gain": 0,
          "subsample": 0.85,
          #'metric':'logloss',
#            'lambda_l1':0.02,
#            'lambda_l2':0.02,
          'seed':666,
          'num_class':6
          }

In [ ]:
loss_list = []
pred = pd.DataFrame()
for i in range(20):
    print('***********第%s次抽样***********'%i)
    x0_part = x0.copy()
    x0_part.index = range(x0_part.shape[0])
    x1.index = range(x1.shape[0])
    permutation = np.random.permutation(x0_part.shape[0])
    
    random.seed(i)
    ratio = int(x0.shape[0]*random.uniform(1.5, 4)/10)

    select = x0_part.iloc[permutation[:ratio], :]
    
    print('负样本抽样数量：')
    print(select.shape)
    
    data_new = pd.concat([x1, select], axis = 0)
    
    permutation = np.random.permutation(data_new.shape[0])

    data_new = data_new.iloc[permutation, :]
    
    label_new = data_new['label'].values
    
    data_new = data_new.drop(['label', 'id'], axis = 1)
    
    dtrain = lgb.Dataset(data_new, label=label_new)

    hist = lgb.cv(params, dtrain, num_boost_round = 1000, verbose_eval=20, early_stopping_rounds=30, stratified=False)
    
    print('最优结果%s'%hist['multi_logloss-mean'][-1])
    loss_list.append(hist['multi_logloss-mean'][-1])
    print('总loss%s'%loss_list)
    model = lgb.train(params=params, train_set=dtrain, num_boost_round=int(1.1*len(hist['multi_logloss-mean'])), verbose_eval=10,\
                      valid_sets=dtrain)
    
    pred_temp = pd.DataFrame(model.predict(test_FE))
    
    pred = pd.concat([pred, pred_temp], axis = 1)

***********第0次抽样***********
负样本抽样数量：
(40279, 1259)
[20]	cv_agg's multi_logloss: 0.522239 + 0.00173581
[20]	cv_agg's multi_logloss: 0.53413 + 0.00105499
[40]	cv_agg's multi_logloss: 0.214341 + 0.000960128
[60]	cv_agg's multi_logloss: 0.104722 + 0.000655142
[80]	cv_agg's multi_logloss: 0.0654616 + 0.000741861
[100]	cv_agg's multi_logloss: 0.0513879 + 0.0008086
[120]	cv_agg's multi_logloss: 0.0463053 + 0.000835119
[140]	cv_agg's multi_logloss: 0.0445138 + 0.000835377
[160]	cv_agg's multi_logloss: 0.0438362 + 0.000909081
[180]	cv_agg's multi_logloss: 0.0436343 + 0.000944185
[200]	cv_agg's multi_logloss: 0.0435945 + 0.00101815
[220]	cv_agg's multi_logloss: 0.0436075 + 0.00107943
[240]	cv_agg's multi_logloss: 0.0436564 + 0.0011532
最优结果0.04357055898915886
总loss[0.02694491490357993, 0.04357055898915886]
[10]	training's multi_logloss: 0.909304
[20]	training's multi_logloss: 0.531743
[30]	training's multi_logloss: 0.32801
[40]	training's multi_logloss: 0.210631
[50]	training's multi_logloss: 0.1

In [30]:
p = pd.DataFrame(sub_sample.file_id)

p['prob0'] = pred[0].mean(axis = 1)
p['prob1'] = pred[1].mean(axis = 1)
p['prob2'] = pred[2].mean(axis = 1)
p['prob3'] = pred[3].mean(axis = 1)
p['prob4'] = pred[4].mean(axis = 1)
p['prob5'] = pred[5].mean(axis = 1)

In [31]:
p.mean()

file_id    26546.000000
prob0          0.899607
prob1          0.007780
prob2          0.014583
prob3          0.012210
prob4          0.001655
prob5          0.064165
dtype: float64

In [32]:
p.to_csv('/home/libo/Security/sub/8.24_bagging_v1.csv', index = None ,encoding = 'utf-8')

In [26]:
1

1

In [36]:
train_list_use = ['lgb_train_v1.csv', 'lgb_train_v2.csv', 'lgb_train_v3.csv']\
+ ['xgb_train_v1.csv', 'xgb_train_v3.csv']\
+['NN_train_v1.csv', 'NN_train_v2.csv', 'AE_type1_train_v1.csv', 'AE_type1_train_v3.csv']\
+['CNN_api_train_v1.csv'] + ['dabai_train.csv']

test_list_use = ['lgb_test_v1.csv', 'lgb_test_v2.csv', 'lgb_test_v3.csv'] + ['xgb_test_v1.csv', 'xgb_test_v3.csv']\
+['NN_test_v1.csv', 'NN_test_v2.csv', 'AE_type1_test_v1.csv', 'AE_type1_test_v3.csv']\
+['CNN_api_test_v1.csv'] + ['dabai_test.csv']

In [37]:
train_meta = pd.DataFrame()
for i in train_list_use:
    print(i)
    train_meta = pd.concat([train_meta, pd.read_csv(i)], axis = 1)
    print(train_meta.shape)

lgb_train_v1.csv
(116624, 6)
lgb_train_v2.csv
(116624, 12)
lgb_train_v3.csv
(116624, 18)
xgb_train_v1.csv
(116624, 24)
xgb_train_v3.csv
(116624, 30)
NN_train_v1.csv
(116624, 36)
NN_train_v2.csv
(116624, 42)
AE_type1_train_v1.csv
(116624, 48)
AE_type1_train_v3.csv
(116624, 54)
CNN_api_train_v1.csv
(116624, 60)
dabai_train.csv
(116624, 150)


In [38]:
test_meta = pd.DataFrame()
for i in test_list_use:
    print(i)
    try:
        x = pd.read_csv(i).drop('file_id', axis = 1)
    except:
        x = pd.read_csv(i)
    test_meta = pd.concat([test_meta, x], axis = 1)
    print(test_meta.shape)

lgb_test_v1.csv
(53093, 6)
lgb_test_v2.csv
(53093, 12)
lgb_test_v3.csv
(53093, 18)
xgb_test_v1.csv
(53093, 24)
xgb_test_v3.csv
(53093, 30)
NN_test_v1.csv
(53093, 36)
NN_test_v2.csv
(53093, 42)
AE_type1_test_v1.csv
(53093, 48)
AE_type1_test_v3.csv
(53093, 54)
CNN_api_test_v1.csv
(53093, 60)
dabai_test.csv
(53093, 150)


In [39]:
test_meta = test_meta[train_meta.columns]
test_meta.columns == train_meta.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [40]:
train_meta.shape, test_meta.shape

((116624, 150), (53093, 150))

In [41]:
train_FE = train_meta.copy()
test_FE = test_meta.copy()

In [42]:
train_FE.shape, test_FE.shape

((116624, 150), (53093, 150))

In [43]:
train_FE['label'] = label

In [46]:
x0 = train_FE[train_FE['label'] == 0]

x1 = train_FE[train_FE['label'] != 0]

x0.shape, x1.shape

((111545, 151), (5079, 151))

In [47]:
index = pd.read_csv('/home/libo/Security/index.csv')

In [49]:
params = {"objective": "multiclass",
          "boosting_type": "gbdt",
          "learning_rate": 0.05,
          "num_leaves": 3,
          # "max_bin": 128,
          "feature_fraction": 0.85,
#           "min_child_samples": 10,
#           "min_child_weight": 150,
#           "min_split_gain": 0,
          "subsample": 0.85,
          #'metric':'logloss',
#            'lambda_l1':0.02,
#            'lambda_l2':0.02,
          'seed':666,
          'num_class':6
          }

In [ ]:
loss_list = []
pred = pd.DataFrame()
for i in range(20):
    print('***********第%s次抽样***********'%i)
    x0_part = x0.copy()
    x0_part.index = range(x0_part.shape[0])
    x1.index = range(x1.shape[0])
    permutation = np.random.permutation(x0_part.shape[0])
    
    random.seed(i)
    ratio = int(x0.shape[0]*random.uniform(1.5, 4)/10)

    select = x0_part.iloc[permutation[:ratio], :]
    
    print('负样本抽样数量：')
    print(select.shape)
    
    data_new = pd.concat([x1, select], axis = 0)
    
    permutation = np.random.permutation(data_new.shape[0])

    data_new = data_new.iloc[permutation, :]
    
    label_new = data_new['label'].values
    try:
        data_new = data_new.drop(['label', 'id'], axis = 1)
    except:
        data_new = data_new.drop(['label'], axis = 1)
    dtrain = lgb.Dataset(data_new, label=label_new)

    hist = lgb.cv(params, dtrain, num_boost_round = 1000, verbose_eval=20, early_stopping_rounds=30, stratified=False)
    
    print('最优结果%s'%hist['multi_logloss-mean'][-1])
    loss_list.append(hist['multi_logloss-mean'][-1])
    print('总loss%s'%loss_list)
    model = lgb.train(params=params, train_set=dtrain, num_boost_round=int(1.1*len(hist['multi_logloss-mean'])), verbose_eval=10,\
                      valid_sets=dtrain)
    
    pred_temp = pd.DataFrame(model.predict(test_FE))
    
    pred = pd.concat([pred, pred_temp], axis = 1)

***********第0次抽样***********
负样本抽样数量：
(40279, 151)
[20]	cv_agg's multi_logloss: 0.522343 + 0.00170798
[40]	cv_agg's multi_logloss: 0.199545 + 0.0025768
[60]	cv_agg's multi_logloss: 0.0886813 + 0.00301587
[80]	cv_agg's multi_logloss: 0.0489693 + 0.00331693
[100]	cv_agg's multi_logloss: 0.0347014 + 0.00358443
[120]	cv_agg's multi_logloss: 0.0295438 + 0.00380038
[140]	cv_agg's multi_logloss: 0.0276366 + 0.00389922
[160]	cv_agg's multi_logloss: 0.0269312 + 0.00398551
[180]	cv_agg's multi_logloss: 0.0266648 + 0.00406272
[200]	cv_agg's multi_logloss: 0.026506 + 0.00405759
[220]	cv_agg's multi_logloss: 0.0264162 + 0.00404613
[240]	cv_agg's multi_logloss: 0.0263772 + 0.00404248
[260]	cv_agg's multi_logloss: 0.0263699 + 0.00403675
[280]	cv_agg's multi_logloss: 0.0263662 + 0.00404325
最优结果0.02635044486702374
总loss[0.02635044486702374]
[10]	training's multi_logloss: 0.901764
[20]	training's multi_logloss: 0.521359
[30]	training's multi_logloss: 0.316132
[40]	training's multi_logloss: 0.197981
[50]	

In [51]:
pred

,0,1,2,3,4,5,0,1,2,3,...,2,3,4,5,0,1,2,3,4,5
0,0.999387,0.000056,0.000109,0.000031,0.000033,0.000384,0.998805,0.000090,0.000237,0.000038,...,0.000284,0.000058,0.000040,0.000727,0.999127,0.000058,0.000172,0.000051,0.000035,0.000558
1,0.998987,0.000030,0.000738,0.000012,0.000029,0.000203,0.998670,0.000053,0.000729,0.000023,...,0.000797,0.000026,0.000044,0.000308,0.999129,0.000031,0.000554,0.000015,0.000030,0.000242
2,0.999899,0.000006,0.000042,0.000005,0.000005,0.000044,0.999792,0.000011,0.000084,0.000010,...,0.000086,0.000011,0.000013,0.000100,0.999846,0.000010,0.000061,0.000008,0.000007,0.000069
3,0.999624,0.000027,0.000112,0.000047,0.000019,0.000170,0.999376,0.000040,0.000208,0.000051,...,0.000177,0.000051,0.000022,0.000343,0.999627,0.000025,0.000110,0.000047,0.000019,0.000171
4,0.999921,0.000006,0.000020,0.000005,0.000005,0.000044,0.999825,0.000011,0.000051,0.000010,...,0.000066,0.000011,0.000013,0.000100,0.999883,0.000008,0.000034,0.000007,0.000006,0.000061
5,0.999646,0.000011,0.000188,0.000008,0.000009,0.000138,0.999131,0.000026,0.000525,0.000020,...,0.000551,0.000023,0.000025,0.000260,0.999374,0.000016,0.000304,0.000013,0.000012,0.000281
6,0.999756,0.000038,0.000074,0.000010,0.000010,0.000113,0.999470,0.000074,0.000206,0.000021,...,0.000166,0.000019,0.000020,0.000160,0.999740,0.000047,0.000092,0.000011,0.000010,0.000100
7,0.999911,0.000006,0.000030,0.000005,0.000005,0.000044,0.999800,0.000011,0.000075,0.000010,...,0.000095,0.000012,0.000013,0.000100,0.999856,0.000009,0.000055,0.000007,0.000007,0.000066
8,0.999153,0.000024,0.000407,0.000043,0.000024,0.000349,0.997786,0.000066,0.001193,0.000072,...,0.001764,0.000109,0.000074,0.000806,0.998645,0.000036,0.000727,0.000057,0.000029,0.000507
9,0.992610,0.000246,0.005882,0.000258,0.000080,0.000923,0.991630,0.000333,0.006142,0.000491,...,0.006003,0.000643,0.000126,0.001101,0.994045,0.000222,0.003780,0.000453,0.000076,0.001424


In [52]:
p = pd.DataFrame(sub_sample.file_id)

p['prob0'] = pred[0].mean(axis = 1)
p['prob1'] = pred[1].mean(axis = 1)
p['prob2'] = pred[2].mean(axis = 1)
p['prob3'] = pred[3].mean(axis = 1)
p['prob4'] = pred[4].mean(axis = 1)
p['prob5'] = pred[5].mean(axis = 1)

In [54]:
p.mean()

file_id    26546.000000
prob0          0.899657
prob1          0.007737
prob2          0.014635
prob3          0.012296
prob4          0.001661
prob5          0.064014
dtype: float64

In [56]:
x = pd.read_csv('/home/libo/Security/sub/8.24_bagging_v1.csv')

In [60]:
x.mean()

file_id    26546.000000
prob0          0.899607
prob1          0.007780
prob2          0.014583
prob3          0.012210
prob4          0.001655
prob5          0.064165
dtype: float64

In [61]:
np.corrcoef([p['prob0'], x['prob0']], rowvar=1)

array([[1.       , 0.9997724],
       [0.9997724, 1.       ]])

In [62]:
pred = x.copy()

In [63]:
pred['prob0'] = 0.5*x['prob0'] + 0.5*p['prob0']
pred['prob1'] = 0.5*x['prob1'] + 0.5*p['prob1']
pred['prob2'] = 0.5*x['prob2'] + 0.5*p['prob2']
pred['prob3'] = 0.5*x['prob3'] + 0.5*p['prob3']
pred['prob4'] = 0.5*x['prob4'] + 0.5*p['prob4']
pred['prob5'] = 0.5*x['prob5'] + 0.5*p['prob5']

In [64]:
pred.mean()

file_id    26546.000000
prob0          0.899632
prob1          0.007758
prob2          0.014609
prob3          0.012253
prob4          0.001658
prob5          0.064089
dtype: float64

In [67]:
pred.to_csv('/home/libo/Security/sub/8.25_bagging_v1_v2.csv', index = None ,encoding = 'utf-8')

In [69]:
sub_sample = pd.read_csv('/home/libo/Security/3rd_security_submit_sample.csv')

label = pd.read_pickle('/home/libo/Security/label.pkl')
label = label[0].values

feature = pd.read_pickle('/home/libo/Security/top_feature1000.pkl')

feature = feature.iloc[:,:301]

feature['id'] = feature.iloc[:, 0].values

feature.drop('file_id', axis = 1, inplace = True)

train_FE = feature.iloc[:len(label),:]
test_FE = feature.iloc[len(label):,:]

train_FE.shape, test_FE.shape

((116624, 300), (53093, 300))

In [70]:
path = '/home/libo/Security/stack'
os.chdir(path)

list_ = os.listdir()
train_list = [x for x in list_ if 'train' in x]
test_list = [x for x in list_ if 'test' in x]

len(train_list) == len(test_list)

True

In [71]:
len(train_list)

31

In [72]:
train_meta = pd.DataFrame()
for i in train_list:
    train_meta = pd.concat([train_meta, pd.read_csv(i)], axis = 1)
    print(train_meta.shape)

(116624, 6)
(116624, 12)
(116624, 18)
(116624, 24)
(116624, 30)
(116624, 36)
(116624, 42)
(116624, 48)
(116624, 54)
(116624, 60)
(116624, 66)
(116624, 72)
(116624, 78)
(116624, 84)
(116624, 90)
(116624, 96)
(116624, 102)
(116624, 108)
(116624, 114)
(116624, 204)
(116624, 210)
(116624, 216)
(116624, 222)
(116624, 228)
(116624, 234)
(116624, 240)
(116624, 246)
(116624, 252)
(116624, 258)
(116624, 264)
(116624, 270)


In [73]:
test_meta = pd.DataFrame()
for i in test_list:
    try:
        x = pd.read_csv(i).drop('file_id', axis = 1)
    except:
        x = pd.read_csv(i)
    test_meta = pd.concat([test_meta, x], axis = 1)
    print(test_meta.shape)

(53093, 6)
(53093, 12)
(53093, 18)
(53093, 24)
(53093, 30)
(53093, 36)
(53093, 42)
(53093, 48)
(53093, 54)
(53093, 60)
(53093, 66)
(53093, 72)
(53093, 78)
(53093, 84)
(53093, 90)
(53093, 96)
(53093, 102)
(53093, 108)
(53093, 114)
(53093, 120)
(53093, 126)
(53093, 132)
(53093, 138)
(53093, 144)
(53093, 150)
(53093, 156)
(53093, 162)
(53093, 168)
(53093, 174)
(53093, 180)
(53093, 270)


In [74]:
test_meta = test_meta[train_meta.columns]
test_meta.columns == train_meta.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [75]:
train_meta.shape, test_meta.shape

((116624, 270), (53093, 270))

In [76]:
test_FE.index = range(test_FE.shape[0])
test_meta.index = range(test_meta.shape[0])

In [83]:
params = {"objective": "multiclass",
          "boosting_type": "gbdt",
          "learning_rate": 0.05,
          "num_leaves": 15,
          # "max_bin": 128,
          "feature_fraction": 0.85,
#           "min_child_samples": 10,
#           "min_child_weight": 150,
#           "min_split_gain": 0,
          "subsample": 0.85,
          #'metric':'logloss',
           'lambda_l1':2,
           'lambda_l2':2,
          'seed':666,
          'num_class':6
          }

In [78]:
train_FE['label'] = label

test_FE.drop('id', axis = 1, inplace = True)

train_FE.shape, test_FE.shape

/home/libo/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/libo/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


((116624, 301), (53093, 299))

In [79]:
train_FE = pd.concat([train_FE, train_meta], axis = 1)
test_FE = pd.concat([test_FE, test_meta], axis = 1)

train_FE.shape, test_FE.shape

((116624, 571), (53093, 569))

In [80]:
np.sum(train_FE.drop(['id', 'label'], axis = 1).columns == test_FE.columns)

569

In [81]:
x0 = train_FE[train_FE['label'] == 0]

x1 = train_FE[train_FE['label'] != 0]

x0.shape, x1.shape

((111545, 571), (5079, 571))

In [82]:
index = pd.read_csv('/home/libo/Security/index.csv')

In [ ]:
loss_list = []
pred = pd.DataFrame()
for i in range(20):
    print('***********第%s次抽样***********'%i)
    x0_part = x0.copy()
    x0_part.index = range(x0_part.shape[0])
    x1.index = range(x1.shape[0])
    permutation = np.random.permutation(x0_part.shape[0])
    
    random.seed(i+10086)
    ratio = int(x0.shape[0]*random.uniform(1.5, 4)/10)

    select = x0_part.iloc[permutation[:ratio], :]
    
    print('负样本抽样数量：')
    print(select.shape)
    
    data_new = pd.concat([x1, select], axis = 0)
    
    permutation = np.random.permutation(data_new.shape[0])

    data_new = data_new.iloc[permutation, :]
    
    label_new = data_new['label'].values
    
    data_new = data_new.drop(['label', 'id'], axis = 1)
    
    dtrain = lgb.Dataset(data_new, label=label_new)

    hist = lgb.cv(params, dtrain, num_boost_round = 1000, verbose_eval=20, early_stopping_rounds=30, stratified=False)
    
    print('最优结果%s'%hist['multi_logloss-mean'][-1])
    loss_list.append(hist['multi_logloss-mean'][-1])
    print('总loss%s'%loss_list)
    model = lgb.train(params=params, train_set=dtrain, num_boost_round=int(1.1*len(hist['multi_logloss-mean'])), verbose_eval=10,\
                      valid_sets=dtrain)
    
    pred_temp = pd.DataFrame(model.predict(test_FE))
    
    pred = pd.concat([pred, pred_temp], axis = 1)

***********第0次抽样***********
负样本抽样数量：
(17946, 571)
[20]	cv_agg's multi_logloss: 0.536136 + 0.00328952
[40]	cv_agg's multi_logloss: 0.216745 + 0.00440766
[60]	cv_agg's multi_logloss: 0.107337 + 0.00509375
[80]	cv_agg's multi_logloss: 0.0685143 + 0.00557223
[100]	cv_agg's multi_logloss: 0.054815 + 0.00584947
[120]	cv_agg's multi_logloss: 0.0501334 + 0.00602871
[140]	cv_agg's multi_logloss: 0.0487183 + 0.00617431
[160]	cv_agg's multi_logloss: 0.0485376 + 0.00640683
[180]	cv_agg's multi_logloss: 0.0487981 + 0.00664318
最优结果0.04852498567484852
总loss[0.04852498567484852]
[10]	training's multi_logloss: 0.909154
[20]	training's multi_logloss: 0.53043
[30]	training's multi_logloss: 0.32542
[40]	training's multi_logloss: 0.206719
[50]	training's multi_logloss: 0.135874
[60]	training's multi_logloss: 0.0927706
[70]	training's multi_logloss: 0.0660614
[80]	training's multi_logloss: 0.0493261
[90]	training's multi_logloss: 0.0384006
[100]	training's multi_logloss: 0.0309029
[110]	training's multi_log

In [87]:
pred.shape

(53093, 120)

In [89]:
p = pd.DataFrame(sub_sample.file_id)

p['prob0'] = pred[0].mean(axis = 1)
p['prob1'] = pred[1].mean(axis = 1)
p['prob2'] = pred[2].mean(axis = 1)
p['prob3'] = pred[3].mean(axis = 1)
p['prob4'] = pred[4].mean(axis = 1)
p['prob5'] = pred[5].mean(axis = 1)

In [90]:
p.mean()

file_id    26546.000000
prob0          0.899965
prob1          0.007818
prob2          0.014223
prob3          0.012386
prob4          0.001676
prob5          0.063932
dtype: float64

In [91]:
p.to_csv('/home/libo/Security/sub/8.26_bagging_v3.csv', index = None ,encoding = 'utf-8')

In [99]:
x1 = pd.read_csv('/home/libo/Security/sub/8.25_bagging_v1_v2.csv')

In [101]:
pred = p.copy()

In [102]:
pred['prob0'] = 0.7*x1['prob0'] + 0.3*p['prob0']
pred['prob1'] = 0.7*x1['prob1'] + 0.3*p['prob1']
pred['prob2'] = 0.7*x1['prob2'] + 0.3*p['prob2']
pred['prob3'] = 0.7*x1['prob3'] + 0.3*p['prob3']
pred['prob4'] = 0.7*x1['prob4'] + 0.3*p['prob4']
pred['prob5'] = 0.7*x1['prob5'] + 0.3*p['prob5']

In [105]:
pred.to_csv('/home/libo/Security/sub/8.26_bagging_v1_v2_v3.csv', index = None ,encoding = 'utf-8')

In [106]:
pred

,file_id,prob0,prob1,prob2,prob3,prob4,prob5
0,0,0.998841,0.000110,0.000219,0.000105,0.000066,0.000659
1,1,0.998663,0.000083,0.000760,0.000077,0.000066,0.000351
2,2,0.999697,0.000039,0.000089,0.000040,0.000027,0.000108
3,3,0.999208,0.000073,0.000206,0.000082,0.000044,0.000388
4,4,0.999716,0.000036,0.000078,0.000038,0.000029,0.000104
5,5,0.999161,0.000053,0.000477,0.000054,0.000041,0.000214
6,6,0.999500,0.000074,0.000174,0.000054,0.000037,0.000161
7,7,0.999707,0.000043,0.000082,0.000037,0.000029,0.000102
8,8,0.997921,0.000128,0.000861,0.000151,0.000121,0.000817
9,9,0.992870,0.000318,0.004637,0.001034,0.000193,0.000949


##### xgb bagging

In [109]:
x0 = train_FE[train_FE['label'] == 0]

x1 = train_FE[train_FE['label'] != 0]

x0.shape, x1.shape

((111545, 571), (5079, 571))

In [112]:
params = {'learning_rate' :  0.05, 'n_estimators':3000, 'max_depth': 3, 'num_class' : 6,  
              'subsample':  0.85, 'colsample_bytree':  0.85,'seed':100,
             'eta': 0.05,'alpha': 0, 'silent': 1, 'objective': 'multi:softprob', 'booster': 'gbtree',
            'lambda': 0, 'eval_metric': "mlogloss"}

In [ ]:
loss_list = []
pred = pd.DataFrame()
for i in range(20):
    print('***********第%s次抽样***********'%i)
    x0_part = x0.copy()
    x0_part.index = range(x0_part.shape[0])
    x1.index = range(x1.shape[0])
    permutation = np.random.permutation(x0_part.shape[0])
    
    random.seed(10*i+2018)
    
    ratio = int(x0.shape[0]*random.uniform(1.5,5)/10)

    select = x0_part.iloc[permutation[:ratio], :]
    
    print('负样本抽样数量：')
    print(select.shape)
    
    data_new = pd.concat([x1, select], axis = 0)
    
    permutation = np.random.permutation(data_new.shape[0])

    data_new = data_new.iloc[permutation, :]
    
    label_new = data_new['label'].values
    
    data_new = data_new.drop(['label', 'id'], axis = 1)
    
    dtrain = xgb.DMatrix(data_new, label=label_new)
    
    devaluation = xgb.DMatrix(test_FE)
    
    hist = xgb.cv(dtrain=dtrain, early_stopping_rounds=30, stratified=False, num_boost_round=1000, params=params, 
       verbose_eval=20)    
    print('最优结果%s'%hist['test-mlogloss-mean'].values[-1])
    loss_list.append(hist['test-mlogloss-mean'].values[-1])
    print('总loss%s'%loss_list)
    
    evallist = [(dtrain, 'train_data')]
    
    model = xgb.train(params=params, dtrain=dtrain, verbose_eval=20, evals=evallist, \
                      num_boost_round=int(1.1*(len(hist['test-mlogloss-mean']))))
    
    pred_temp = pd.DataFrame(model.predict(devaluation))
    
    pred = pd.concat([pred, pred_temp], axis = 1)

***********第0次抽样***********
负样本抽样数量：
(37546, 571)
[0]	train-mlogloss:1.64588+5.96955e-05	test-mlogloss:1.64621+7.02772e-05


In [118]:
pred.shape

(53093, 120)

In [119]:
p = pd.DataFrame(sub_sample.file_id)

p['prob0'] = pred[0].mean(axis = 1)
p['prob1'] = pred[1].mean(axis = 1)
p['prob2'] = pred[2].mean(axis = 1)
p['prob3'] = pred[3].mean(axis = 1)
p['prob4'] = pred[4].mean(axis = 1)
p['prob5'] = pred[5].mean(axis = 1)

In [121]:
p.mean()

file_id    26545.550781
prob0          0.900738
prob1          0.007801
prob2          0.014163
prob3          0.011954
prob4          0.001739
prob5          0.063801
dtype: float32

In [122]:
p.to_csv('/home/libo/Security/sub/8.27_xgb_bagging_v1.csv', index = None ,encoding = 'utf-8')